In [82]:
import pysd
import pandas as pd
import numpy as np

In [97]:
def calculate_profit_index(scenario_df):
    # Load the base scenario CSV file
    base_df = pd.read_csv('base_scenario.csv')

    # Merge base and scenario data on 'time' column
    merged_df = pd.merge(base_df, scenario_df, on='time')

    # Calculate Total Profit Index for the scenario
    merged_df['Total_Profit_Index'] = (merged_df['total profit'] - merged_df['base total profit'])

    return merged_df['Total_Profit_Index'].sum()


In [98]:
model = pysd.read_vensim('innovation sd.mdl')

In [126]:
# Fixed parameter values
fixed_params = {
    'hd': 6,
    'td': 10,
    'hrec': 4,
    '"%reduction of demand"': 0.5,
    'k': 1,
    'km1': 0,
    'km2': 0,
    'km3': 0,
    'km4': 0,
    'km5': 1,
    '"kused-prod"': 1,
    'krmsi': 1,
    # Add more fixed parameters if needed
}

# Define the parameter values to iterate
parameter_vals = {
    'kp': [0, 1],
    'kppi': [0, 1],
    'kd': [0, 1],
    'krpi': [0, 1],
    'kc': [0, 1],
    'kci': [0, 1],
}

In [128]:
import itertools

best_profit_index = None
worst_profit_index = None
best_scenario = None
worst_scenario = None
best_scenario_index = None
worst_scenario_index  = None

# Loop through each scenario
for scenario in itertools.product(*parameter_vals.values()):
    scenario_dict = {**fixed_params, **{param: value for param, value in zip(parameter_vals.keys(), scenario)}}

    # Run the simulation with the current scenario parameter values
    result = model.run(params=scenario_dict, return_columns=['total profit'])
    result = result.rename_axis('time')
    # Calculate the profit index and bounds for the current scenario
    profit_index = calculate_profit_index(result)
    # Update the best and worst average bounds
    if best_profit_index is None or profit_index > best_profit_index:
        best_profit_index = profit_index
        best_scenario = result
        best_scenario_index  = scenario

    if worst_profit_index is None or profit_index < worst_profit_index:
        worst_profit_index = profit_index
        worst_scenario = result
        worst_scenario_index = scenario

# Print the best and worst scenarios
print("Best Scenario:")
print(best_scenario)
print("---")
print("Worst Scenario:")
print(worst_scenario)

Best Scenario:
      total profit
time              
0.0   0.000000e+00
0.5  -4.140000e+05
1.0  -6.384000e+05
1.5  -9.327188e+05
2.0  -1.245616e+06
...            ...
70.0  3.741192e+07
70.5  3.750657e+07
71.0  3.761058e+07
71.5  3.776085e+07
72.0  3.789172e+07

[145 rows x 1 columns]
---
Worst Scenario:
      total profit
time              
0.0   0.000000e+00
0.5  -4.140000e+05
1.0  -6.384000e+05
1.5  -9.327188e+05
2.0  -1.245616e+06
...            ...
70.0  2.493569e+07
70.5  2.501909e+07
71.0  2.507347e+07
71.5  2.513075e+07
72.0  2.524573e+07

[145 rows x 1 columns]


In [129]:
best_profit_index

1073097783.2738167

In [130]:
worst_profit_index

74227478.49634561